# Regcoil
Stellacode includes the [regcoil](https://github.com/landreman/regcoil) algorithm for finding the optimal current density for reproducing a given magnetic field from a given coil surface.

The first step is to choose a plasma configuration and create the regcoil solver object. Some predefined configuration are available in stellacode. Let's choose the W7X configuration.

In [1]:
from mayavi import mlab
mlab.init_notebook()

from stellacode.definitions import w7x_plasma
from stellacode.costs import EMCost
from stellacode.surface import IntegrationParams

em_cost = EMCost.from_plasma_config(
    plasma_config=w7x_plasma,
    integration_par=IntegrationParams(num_points_u=16, num_points_v=16),
)


Notebook initialized with ipy backend.


I0000 00:00:1696860086.243532   17086 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Then we can build a coil factory (a class used to create coil objects):

In [2]:
from stellacode.surface import WrappedCoil

coil_factory = WrappedCoil.from_plasma(
    surf_plasma=em_cost.Sp,
    surf_type="toroidal",
    n_harmonics=4,
    factor=6,
)



Then we can build the coil operator, which is a linear operator acting on current parameters and returning a current object:

In [3]:
coil_op = coil_factory()

We can retrieve the current parameters and use them to build a coil surface:

Then we can solve for the value of $\lambda$ given to EMCost

In [4]:
cost, metrics, results, S = em_cost.cost(coil_op)
print(metrics)

{'err_max_B': Array(2.1029208, dtype=float32), 'cost_B': Array(48.3173, dtype=float32), 'em_cost': Array(1.0466975e+14, dtype=float32), 'cost_J': Array(1.04669756e+15, dtype=float32), 'max_j': Array(2714388.2, dtype=float32)}


In [5]:
coil_surface = coil_op.get_coil(results.phi_mn_wnet_cur) 

Then we can plot the current obtained:

In [6]:

em_cost.Sp.plot()
coil_surface.plot(vector_field=coil_surface.j_3d)